In [7]:
import os
import json
import pandas as pd

In [8]:
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("GOOGLE_API_KEY") 


In [9]:
from langchain.llms import GooglePalm
llm = GooglePalm(google_api_key=KEY, temperature=0.7)

In [10]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [11]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your responses like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
"""

In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "5":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },

}

In [14]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [15]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [16]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [17]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [19]:
filePath = "I:\Z\Data Science\AI\GEN AI\mcqgenerator\data.txt"

In [20]:
with open(filePath, 'r') as file:
    TEXT = file.read()

In [21]:
TEXT

"Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]\n\nFrom a t

In [22]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [23]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [24]:
response = generate_evaluate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field

In [64]:
# print(response)
quiz = response.get("quiz")
quiz = quiz.replace('```json\n', '')
quiz = quiz.replace("```",'')
print(type(quiz))


<class 'str'>


In [65]:
quiz = json.loads(quiz)


<class 'dict'>


In [59]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [ ]:
quiz_table_data

[{'MCQ': 'multiple choice question',
  'Choices': 'a: choice here | b: choice here | c: choice here | d: choice here',
  'Correct': 'correct answer'},
 {'MCQ': 'multiple choice question',
  'Choices': 'a: choice here | b: choice here | c: choice here | d: choice here',
  'Correct': 'correct answer'},
 {'MCQ': 'multiple choice question',
  'Choices': 'a: choice here | b: choice here | c: choice here | d: choice here',
  'Correct': 'correct answer'},
 {'MCQ': 'multiple choice question',
  'Choices': 'a: choice here | b: choice here | c: choice here | d: choice here',
  'Correct': 'correct answer'},
 {'MCQ': 'multiple choice question',
  'Choices': 'a: choice here | b: choice here | c: choice here | d: choice here',
  'Correct': 'correct answer'}]

In [67]:
quiz=pd.DataFrame(quiz_table_data)

In [69]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')